# 6. SVM (Support Vector Machine)
분류 모델 중 하나로 벡터(Vector) 개념을 가져와서 사용한다. 데이터프레임의 각 열이 고유의 축을 가지는 벡터공간을 만들고, 각각의 개별의 모든 속성이 축의 좌료로 표시되어 벡터 공간에서 위치를 나타낸다. 속성(열)이 2개가 존재하면 2차원, 4개가 존재하면 4차원 평면 공간에 데이터 셋이 좌표로 표시된다.  

같은 분류 값인 데이터끼리 같은 공간에 위치시켜 벡터 공간을 여러 조각으로 나누고, 새로운 데이터에 대하여 어느 공간에 위치하는지 분류할 수 있게 한다.  
> 그래서 분류. classification

앞에서 반복했던 코드들은 빠르게 넘어가자

In [1]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')

#NaN값이 많은 deck 열을 삭제, embarked와 내용이 겹치는 embark_town 열을 삭제
rdf = df.drop(['deck','embark_town'], axis=1) # axis=1 은 열에 대한 작용이라는 뜻

# age 열에 나이 데이터가 없는 모든 행을 삭제 - age 열(891개 중 177개의 NaN 값)
rdf = df.dropna(subset=['age'], how='any', axis=0) # any: 여기선 row에 NaN 값이 하나라도 있으면 drop

# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
rdf['embarked'].fillna(most_freq, inplace=True)
# 이 코드는 이해 못하겠다 그냥 외우자


ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)
# concat : 데이터를 연결하는 함수. axis 옵션이 주어지면 해당 인덱스에 맞게 옆으로 데이터를 붙일 수 있다
# pd.get_dummies : 서로 무관한 수, 가변수로 더미를 채우는 함수 / prefix : 생성할 dummies_variable의 column 이름 앞에 붙을 prefix를 지정할 수 있습니다.


ndf.drop(['sex','embarked'], axis=1, inplace=True)

X = ndf[['pclass','age','sibsp','parch','female','male',
         'town_C','town_Q','town_S']] # 독립 변수 X
y = ndf[['survived']] # 종속 변수 y

from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=10)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [4]:
from sklearn import svm

# kernel 이란? 데이터를 벡터 공간으로 매핑하는 함수
# rbf : radical basis function
svm_model = svm.SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

y_hat = svm_model.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 1 0 0 0 1 0 0 0]
[[0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


위의 결과를 보면 첫 10개 데이터 중에서 8개만 일치한다. 이제 모형의 예측 능력을 지표로 뽑아보자.

In [14]:
### True False 문제이기 때문에 분류문제는 confusion_matrix를 쓴다.
from sklearn import metrics
import numpy as np
svm_matrix = metrics.confusion_matrix(y_test, y_hat)

a = np.array([['TP', 'FP'], ['FN', 'TN']]) # 미생존자가 0(False), 생존자가 1(True)
print(a,'\n = ',svm_matrix)

[['TP' 'FP']
 ['FN' 'TN']] 
 =  [[120   5]
 [ 35  55]]


총 215명의 승객 중  
미생존자를 정확히 예측한 TP = 120명  
미생존자를 생존자로 잘못 분류한 FP = 5명  
생존자를 미생존자로 잘못 분류한 FN = 35명  
생존자를 정확하게 예측한 TN = 55명  
미생존자=0=False, 생존자=1=True

In [15]:
svm_report = metrics.classification_report(y_test, y_hat)
print(svm_report)

              precision    recall  f1-score   support

           0       0.77      0.96      0.86       125
           1       0.92      0.61      0.73        90

    accuracy                           0.81       215
   macro avg       0.85      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215



classification_report()함수로 출력된 precision, recall, f1-score 지표를 보자.  
f1-score 지표를 보면 미생존자(0) 예측 정확도가 0.86이고,  
생존자(1) 예측의 정확도는 0.73으로 예측 능력에 차이가 있다.  
사실 KNN 모형과 예측 능력에는 큰 차이가 없다.

Q1. 그럼 왜 KNN 대신 SVM을 쓰는거지?